In [2]:
#参考了优秀作业，分别在Baseline中利用ALS和SGD
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import numpy as np

In [3]:
# 加载训练集
data1=pd.read_csv('./combined_data_1.txt',sep=' ',header=None)
data2=pd.read_csv('./combined_data_2.txt',sep=' ',header=None)
data3=pd.read_csv('./combined_data_3.txt',sep=' ',header=None)
data4=pd.read_csv('./combined_data_4.txt',sep=' ',header=None)

#处理训练集
def data_cleaning(data):
    data['movie_id'] = data[0].map(lambda x :int(x[:-1]) if ':' in x else np.NAN)
    data = data.fillna(method='ffill')
    data = data[data[0].map(lambda x: ':' not in x)]
    data['user_id'] = data[0].map(lambda x:x.split(',')[0]).astype(np.int)
    data['rating'] = data[0].map(lambda x:x.split(',')[1]).astype(np.int)
    data=data.drop([data.columns[0]],axis=1)

    return data

#处理probe数据函数
def probe_cleaning(data):
    data['movie_id'] = data[0].map(lambda x :int(x[:-1]) if ':' in x else np.NAN)
    data = data.fillna(method='ffill')
    data = data[data[0].map(lambda x: ':' not in x)]
    data.columns = ['user_id','movie_id']
    data.user_id=data.user_id.astype(np.int)
    data = data[['movie_id','user_id']]
    
    return data

data1 = data_cleaning(data1)
data2 = data_cleaning(data2)
data3 = data_cleaning(data3)
data4 = data_cleaning(data4)
data_all = pd.concat([data1,data2,data3,data4],ignore_index=True)

In [1]:
data_all.shape

NameError: name 'data_all' is not defined

In [6]:
#数据集切分，选user_id为5倍数，movie_id为3倍数
data_c = data_all[(data_all.user_id % 5 ==0) & (data_all.movie_id%3 ==0)]
data_c.shape

(6761990, 3)

In [9]:
data_c.movie_id = data_c.movie_id.astype(np.int64)
data_c

C:\Users\Ricardo\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,movie_id,user_id,rating
700,3,1644750,3
702,3,616720,4
705,3,701730,2
706,3,1614320,4
714,3,67315,4
720,3,714960,4
723,3,1632700,4
725,3,79160,4
726,3,1859725,4
735,3,672980,5


In [10]:
#转换surprise格式
reader = Reader(line_format='item user rating',sep=',',skip_lines=1)
data = Dataset.load_from_df(data_c[['movie_id','user_id','rating']],reader=reader)

In [33]:
train_set = data.build_full_trainset()  #全集作为测试集
#SGD算法
bsl_options = {'method':'sgd','n_epochs':5}
algo = BaselineOnly(bsl_options=bsl_options)
kf = KFold(n_splits=3)

In [34]:
for train_set,test_set in kf.split(data):
    #训练并预测
    algo.fit(train_set)
    predictions = algo.test(test_set)
    #计算RMSE
    accuracy.rmse(predictions,verbose=True)

Estimating biases using sgd...
RMSE: 0.9353
Estimating biases using sgd...
RMSE: 0.9356
Estimating biases using sgd...
RMSE: 0.9366


In [15]:
#prob数据处理
probe_data = pd.read_csv('probe.txt',sep=' ',header=None)
probe_data = probe_cleaning(probe_data)

In [16]:
probe_data.shape

(1408395, 2)

In [17]:
probe_data_c = probe_data[(probe_data['user_id'] % 5==0) & (probe_data['movie_id'] % 3 ==0)]

In [20]:
probe_data_c.sort_values(by='movie_id',ascending=False).head()

,movie_id,user_id
564592,17769.0,1021320
564531,17769.0,1741600
564589,17769.0,1693315
564571,17769.0,713715
564529,17769.0,575830


In [27]:
#probe补全成测试集
p_data = pd.merge(probe_data_c,data_c)

#处理成和testset一样的格式
probe_test = []
for i in p_data.values:
    tup=(i[0],i[1],i[2])
    probe_test.append(tup)

probe_prediction = algo.test(probe_test)

In [28]:
accuracy.rmse(probe_prediction,verbose=True)

RMSE: 0.9935


0.9934785207426676

In [29]:
#ALS算法
bsl_options = {'method':'als','n_epochs':5,'reg_u':12,'reg_i':5}
algo = BaselineOnly(bsl_options=bsl_options)
kf = KFold(n_splits=3)
for train_set,test_set in kf.split(data):
    algo.fit(train_set)
    predctions = algo.test(test_set)
    accuracy.rmse(predictions,verbose=True)


Estimating biases using als...
RMSE: 0.9355
Estimating biases using als...
RMSE: 0.9355
Estimating biases using als...
RMSE: 0.9355


In [30]:
prob_prediction = algo.test(probe_test)
accuracy.rmse(prob_prediction,verbose=True)

RMSE: 0.9633


0.9633325731296605

In [37]:
#查看使用NormalPredictor的效果
from surprise import NormalPredictor
algo = NormalPredictor()
kf = KFold(n_splits=3)
for train_set,test_set in kf.split(data):
    algo.fit(train_set)
    predictions = algo.test(test_set)
    accuracy.rmse(predictions,verbose=True)

RMSE: 1.4684
RMSE: 1.4678
RMSE: 1.4676
